# Extreme Gradient Boosting (XGBoost)

In [1]:
import numpy as np
import pandas as pd 
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
from sklearn.preprocessing import scale 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score,roc_curve
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
diabetes = pd.read_csv("diabetes.csv")
df = diabetes.copy()
y= df["Outcome"]
X= df.drop(['Outcome'], axis=1) 
#X= df["Pregnancies"]
X= pd.DataFrame(X)

X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, y, test_size=0.30, random_state=42)

In [7]:
xgb_model = XGBClassifier(eval_metric="mlogloss").fit(X_Train, Y_Train)

In [8]:
xgb_model.get_params()

{'objective': 'binary:logistic',
 'use_label_encoder': True,
 'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'enable_categorical': False,
 'gamma': 0,
 'gpu_id': -1,
 'importance_type': None,
 'interaction_constraints': '',
 'learning_rate': 0.300000012,
 'max_delta_step': 0,
 'max_depth': 6,
 'min_child_weight': 1,
 'missing': nan,
 'monotone_constraints': '()',
 'n_estimators': 100,
 'n_jobs': 4,
 'num_parallel_tree': 1,
 'predictor': 'auto',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'subsample': 1,
 'tree_method': 'exact',
 'validate_parameters': 1,
 'verbosity': None,
 'eval_metric': 'mlogloss'}

### Tahmin

In [9]:
y_pred = xgb_model.predict(X_Test)
accuracy_score(Y_Test, y_pred)

0.7359307359307359

### Model Tuning

In [12]:
xgb_params = {
    'n_estimators' : [100, 500, 1000, 2000],
    'subsample' : [0.6, 0.8, 1.0],
    'max_depth' :[3,4,5,6],
    'learning_rate' : [0.1, 0.01, 0.02, 0.05],
    'min_samples_split' : [2,5,10]
}

In [13]:
xgb = XGBClassifier()
xgb_cv_model=GridSearchCV(xgb, xgb_params, cv=10, n_jobs=-1 ,verbose=2)

In [ ]:
xgb_cv_model.fit(X_Train, Y_Train)

In [ ]:
print("En iyi parametreler:", str(xgb_cv_model.best_params_))

In [23]:
?XGBClassifier

Init signature:
XGBClassifier(
    *,
    objective: Union[str, Callable[[numpy.ndarray, numpy.ndarray], Tuple[numpy.ndarray, numpy.ndarray]], NoneType] = 'binary:logistic',
    use_label_encoder: bool = True,
    **kwargs: Any,
) -> None
Docstring:     
Implementation of the scikit-learn API for XGBoost classification.


Parameters
----------

    n_estimators : int
        Number of boosting rounds.
    use_label_encoder : bool
        (Deprecated) Use the label encoder from scikit-learn to encode the labels. For new
        code, we recommend that you set this parameter to False.

    max_depth :  Optional[int]
        Maximum tree depth for base learners.
    learning_rate : Optional[float]
        Boosting learning rate (xgb's "eta")
    verbosity : Optional[int]
        The degree of verbosity. Valid values are 0 (silent) - 3 (debug).
    objective : typing.Union[str, typing.Callable[[numpy.ndarray, numpy.ndarray], typing.Tuple[numpy.ndarray, numpy.ndarray]], NoneType]
        Sp

In [25]:
xgb = XGBClassifier(eval_metric="mlogloss",
                    learning_rate = 0.01,
                    max_depth =6,
                    #min_samples_split=2,
                    n_estimators=100,
                    subsample=0.8)

In [26]:
xgb_tuned=xgb.fit(X_Train, Y_Train)

In [27]:
y_pred = xgb_tuned.predict(X_Test)
accuracy_score(Y_Test, y_pred)

0.7445887445887446